In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import pytesseract
import cv2
import os

In [3]:
!cd Data/

In [6]:
img_path = r"D:\Projects\Assignmenr\Data\docs-sm\invoice\00920638.jpg"

In [8]:
image = cv2.imread(img_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding for better OCR accuracy
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# Perform OCR using Tesseract
text = pytesseract.image_to_string(gray,lang='eng')

print("Extracted Text:\n", text)

Extracted Text:
 BORRISTON

LABORATORIES, INC.

Boriston Laboratories, ne:
$050 Beech Place
Ternple Hills, Maryland 20748

‘Telephone: 301-099:3536
843

Telex 2

June 28, 1982

Loriitard Inc.

eae INVOICE # ee

420 English Street of z
Greensboro, N.C. 27405 J-A- Fa
fap 9570
Attention: Or. Harry Ntnneneyer “
feet. +

REFERENCE: Purchase on 1saBY
BL REF 223-201-144
INVOICE #: 1446-1

DESCRIPTION 290)

INDUCTION OF HEPATIC MICROSOMAL
ENZYMES WITH COMPOUNDS B~100,

$10,000.00,

B-101, and B-102

REMITTANCE ADRESS:
Borriston Laboratories, Inc.
DYNAMAC BUILDING

11140 Rockville Pike
Rockville, Md. 20748

Attn: B. Belford
Accounting

ses © INVOICE te

‘A Subsidiary of Dynamac Intemational, Inc.

e£90z600



In [9]:
import re

def clean_text(text):
    """Remove extra spaces, newlines, and special characters."""
    text = text.replace("\n", " ")  # Remove newlines
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [10]:
cleaned_text = clean_text(text)

In [11]:
cleaned_text

'BORRISTON LABORATORIES, INC. Boriston Laboratories, ne: $050 Beech Place Ternple Hills, Maryland 20748 ‘Telephone: 301-099:3536 843 Telex 2 June 28, 1982 Loriitard Inc. eae INVOICE # ee 420 English Street of z Greensboro, N.C. 27405 J-A- Fa fap 9570 Attention: Or. Harry Ntnneneyer “ feet. + REFERENCE: Purchase on 1saBY BL REF 223-201-144 INVOICE #: 1446-1 DESCRIPTION 290) INDUCTION OF HEPATIC MICROSOMAL ENZYMES WITH COMPOUNDS B~100, $10,000.00, B-101, and B-102 REMITTANCE ADRESS: Borriston Laboratories, Inc. DYNAMAC BUILDING 11140 Rockville Pike Rockville, Md. 20748 Attn: B. Belford Accounting ses © INVOICE te ‘A Subsidiary of Dynamac Intemational, Inc. e£90z600'

In [14]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")

def extract_entities(text):
    """Extracts named entities using a fine-tuned BERT model"""
    entities = ner_pipeline(text)
    
    extracted_data = {}
    for ent in entities:
        word, label = ent['word'], ent['entity']
        print(word)
        print(" ")
        print(label)
        print('\n')
        
        # if "MISC" in label and "INV" in word:  # Custom Rule for Invoice Number
        #     extracted_data["invoice_number"] = word
        # elif "DATE" in label:
        #     extracted_data["date"] = word
        # elif "MONEY" in label:
        #     extracted_data["amount"] = word
        # elif "ORG" in label:
        #     extracted_data["vendor"] = word
    
    return extracted_data

# Extract Data
invoice_data = extract_entities(cleaned_text)
print(invoice_data)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


IN
 
B-LOC


Boris
 
B-ORG


##ton
 
I-ORG


Laboratories
 
I-ORG


Bee
 
B-LOC


##ch
 
I-LOC


Place
 
I-LOC


Te
 
I-LOC


##rn
 
I-ORG


##ple
 
I-LOC


Hills
 
I-LOC


Maryland
 
B-LOC


Tel
 
B-ORG


Lori
 
B-ORG


##ita
 
I-ORG


##rd
 
I-ORG


Inc
 
I-ORG


English
 
B-LOC


Street
 
I-LOC


Greens
 
B-LOC


##boro
 
I-LOC


N
 
B-LOC


.
 
I-LOC


C
 
I-LOC


A
 
I-ORG


Or
 
B-PER


Harry
 
B-PER


N
 
I-PER


Bo
 
B-ORG


##rri
 
I-ORG


##ston
 
I-ORG


Laboratories
 
I-ORG


,
 
I-ORG


Inc
 
I-ORG


D
 
B-ORG


##Y
 
I-ORG


##NA
 
I-ORG


##MA
 
I-ORG


Rock
 
B-LOC


##ville
 
I-LOC


Pike
 
I-LOC


Rock
 
I-LOC


##ville
 
I-LOC


M
 
B-LOC


B
 
B-ORG


Bel
 
I-ORG


##ford
 
I-ORG


Accounting
 
I-ORG


D
 
B-ORG


##yna
 
I-ORG


##ma
 
I-ORG


##c
 
I-ORG


In
 
I-ORG


##tem
 
I-ORG


##ational
 
I-ORG


,
 
I-ORG


Inc
 
I-ORG


{}


In [13]:
invoice_data

{'vendor': 'Inc'}

In [15]:
import pandas as pd

def extract_table(text):
    # Extract rows matching pattern (Item, Quantity, Price)
    table_pattern = re.findall(r'(\w+)\s+(\d+)\s+\$?([\d,]+\.?\d{0,2})', text)

    # Convert to Pandas DataFrame
    df = pd.DataFrame(table_pattern, columns=["Item", "Quantity", "Price"])
    
    return df.to_dict(orient='records')

invoice_table = extract_table(cleaned_text)
print(invoice_table)

[]


In [16]:
class DataTransformation:
    def __init__(self):
        pass

    def get_images_labels(self,images,label):
        arr = []
        labels = []
        for i in images:
            print("Looping through all images")
            img = cv2.imread(os.path.join(i))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img ,(120,120))
            img1 = img[0:30,0:120]/255
            img2 = img[30:90, 0:60]/255
            img3 = img[30:90, 60:120]/255
            img4 = img[90:120, 0:120]/255

            img = np.array(
                [cv2.resize(img1,(48,48)),
                 cv2.resize(img2,(48,48)),
                 cv2.resize(img3,(48,48)),
                 cv2.resize(img4,(48,48))]
            )

            img_mean = np.mean(img)
            img = img - img_mean
            img = img /np.std(img)
            arr.append(img)
            labels.append(label)
        return [arr,labels]
    

    






In [17]:
transformer = DataTransformation()

In [18]:
from pathlib import Path

data_path = Path(os.path.join(r"D:\Projects\Assignmenr\artifacts\Data\docs-sm"))

In [19]:
data_path

WindowsPath('D:/Projects/Assignmenr/artifacts/Data/docs-sm')

In [27]:
[advertisement, Y_advertisement] = transformer.get_images_labels(list(data_path.glob("invoice/*.*")), 0)

Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
Looping through all images
L

In [28]:
len(advertisement)

275

In [30]:
advertisement[0].shape

(4, 48, 48)

In [32]:
Y_advertisement[0]

0

['0060046962.jpg', '0060051594.jpg', '0060075043.jpg', '0060080760.jpg', '0060082098.jpg', '0060083495.jpg', '0060095587.jpg', '0060095589.jpg', '0060116328.jpg', '0060121960.jpg', '0060201834.jpg', '0060207574.jpg', '0060281129.jpg', '0060284223.jpg', '00621985.jpg', '00920638.jpg', '00920717.jpg', '01145176.jpg', '01398469.jpg', '03638969.jpg', '03660266.jpg', '03700650.jpg', '03724966.jpg', '03750991.jpg', '1000031431.jpg', '10399076.jpg', '10399232.jpg', '11235021_11235022.jpg', '11238840.jpg', '2001205675.jpg', '2001207806_2001207807.jpg', '2001207874.jpg', '2015066906.jpg', '2021636830.jpg', '2023186187.jpg', '2023322384.jpg', '2023550756.jpg', '2023921600.jpg', '2024525606.jpg', '2024525914.jpg', '2025847098.jpg', '2026004159.jpg', '2026486780.jpg', '2026486823.jpg', '2028691757.jpg', '2028691805_2028691806.jpg', '2028694058.jpg', '2028697984.jpg', '2028699251.jpg', '2028704073.jpg', '2028705019.jpg', '2028706274.jpg', '2028706570.jpg', '2028707266.jpg', '2028707700.jpg', '20287

In [1]:
img_path = r"data\test\invoice\00920638.jpg"

from src.pipeline.prediction_pipeline import PredictionPipeline

In [2]:
pipeline = PredictionPipeline()

In [4]:
result = pipeline.run_pipeline(img=img_path)

Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", shape=(4, 48, 48), dtype=float32). Expected shape (None, 4, 48, 48, 1), but input has incompatible shape (4, 48, 48)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(4, 48, 48), dtype=float32)
  • training=False
  • mask=None


In [60]:
def predict(img):
        try:
            model = tf.keras.models.load_model(r"output\model.h5")
            img = cv2.imread(os.path.join(img))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img ,(120,120))
            img1 = img[0:30,0:120]/255
            img2 = img[30:90, 0:60]/255
            img3 = img[30:90, 60:120]/255
            img4 = img[90:120, 0:120]/255

            img = np.array(
                    [cv2.resize(img1,(48,48)),
                    cv2.resize(img2,(48,48)),
                    cv2.resize(img3,(48,48)),
                    cv2.resize(img4,(48,48))]
                )
            # print(img)

            img_mean = np.mean(img)
            img = img - img_mean
            img = img /np.std(img)
            image = np.asarray(img)
            # print(image.shape)
            image = np.expand_dims(image, axis=-1)
            image = np.expand_dims(image, axis=0)
            print(image.shape)
            pred = model.predict(image)
            return pred
        
        except Exception as e:
            print(e)
            return None

In [58]:
import tensorflow as tf
import cv2
import os
import numpy  as np


In [61]:
reposne = predict(img=img_path)

(1, 4, 48, 48, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step


In [62]:
reposne

array([[9.992461e-01, 2.306239e-04, 5.233443e-04]], dtype=float32)

In [65]:
predicted_class = np.argmax(reposne)

In [66]:
predicted_class

np.int64(0)

In [67]:
from src.pipeline.extraction_pipeline import DataExtractionPipeline

In [72]:
extract_pipeline = DataExtractionPipeline(img_path)

In [73]:
resposne = extract_pipeline.run_pipeline()

Extracted Data: BORRISTON

LABORATORIES, INC.

Boriston Laboratories, ne:
$050 Beech Place
Ternple Hills, Maryland 20748

‘Telephone: 301-099:3536
843

Telex 2

June 28, 1982

Loriitard Inc.

eae INVOICE # ee

420 English Street of z
Greensboro, N.C. 27405 J-A- Fa
fap 9570
Attention: Or. Harry Ntnneneyer “
feet. +

REFERENCE: Purchase on 1saBY
BL REF 223-201-144
INVOICE #: 1446-1

DESCRIPTION 290)

INDUCTION OF HEPATIC MICROSOMAL
ENZYMES WITH COMPOUNDS B~100,

$10,000.00,

B-101, and B-102

REMITTANCE ADRESS:
Borriston Laboratories, Inc.
DYNAMAC BUILDING

11140 Rockville Pike
Rockville, Md. 20748

Attn: B. Belford
Accounting

ses © INVOICE te

‘A Subsidiary of Dynamac Intemational, Inc.

e£90z600

Cleaned Data: BORRISTON LABORATORIES, INC. Boriston Laboratories, ne: $050 Beech Place Ternple Hills, Maryland 20748 ‘Telephone: 301-099:3536 843 Telex 2 June 28, 1982 Loriitard Inc. eae INVOICE # ee 420 English Street of z Greensboro, N.C. 27405 J-A- Fa fap 9570 Attention: Or. Harry Ntnneney

In [74]:
image = cv2.imread(img_path)

In [76]:
extract_pipeline = DataExtractionPipeline(image)

In [77]:
extract_pipeline.run_pipeline()

OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object

'NoneType' object has no attribute 'replace'


'{}\n'